In [3]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

llm = Ollama(model="gemma2:2b")

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""


In [9]:
print(len(text))

575


In [4]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
# ## 2nd type of filtered documents
# llm_transformer_filtered = LLMGraphTransformer(
#     llm=llm,
#     allowed_nodes=["Person", "Country", "Organization"],
#     allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
# )
# graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
#     documents
# )

In [6]:
graph = NetworkxEntityGraph()

# Add nodes to the graph
for node in graph_documents[0].nodes:
    graph.add_node(node.id)

# Add edges to the graph
for edge in graph_documents[0].relationships:
    graph._graph.add_edge(
            edge.source.id,
            edge.target.id,
            relation=edge.type,
        )

In [7]:
chain = GraphQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True
)

In [8]:
question = """Who is Marie Curie?"""
chain.run(question)

/home/nikunj/Downloads/IRE/Project/test/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie 

Full Context:
Marie Curie BORN_IN 1867
Marie Curie WON_NOBEL_PRIZE Nobel Prize
Marie Curie WINS_FIRST_NOBEL_PRIZE First Nobel Prize
Marie Curie WON_SECOND_NOBEL_PRIZE Second Nobel Prize
Marie Curie FIRST_WOMAN_TO_BE_PROFESSOR_AT_UNIVERSITY_OF_PARIS University of Paris

> Finished chain.


'Marie Curie was a pioneering physicist and chemist known for her groundbreaking research on radioactivity.  She was the first woman to be appointed as a professor at the University of Paris, the first person to win Nobel Prizes in two different scientific fields (physics and chemistry), and the only person to have won Nobel Prizes in two fields at all times. \n'